In [9]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime
from pandas.tseries.offsets import CustomBusinessDay
import holidays
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
us_bd = CustomBusinessDay(calendar=holidays.US())
look_back = 30

In [11]:
def hist_trace(days=look_back):
    
    data_frames = []
    
    business_dates = pd.date_range(end=datetime.now(), periods=days+1, freq=us_bd)
    
    for date in business_dates:
        date_str = date.strftime('%Y-%m-%d')
        url = f"https://cdn.finra.org/trace/treasury-aggregates/daily/ts-daily-aggregates-{date_str}.xlsx"
        
        try:
            response = requests.get(url)
            response.raise_for_status()  
            df = pd.read_excel(BytesIO(response.content), header=[3, 4])

            # Clean the DataFrame
            df = df.drop(columns=[('Total', 'VWAP')])
            tips_index = df[df[('Category', 'Unnamed: 0_level_1')] == 'TIPS'].index[0]
            df = df.loc[tips_index:].dropna(axis=0)
            df = df[df[('Category', 'Unnamed: 0_level_1')] != 'Total'].reset_index(drop=True)
            df['Date'] = date_str
            
            data_frames.append(df)
        
        except requests.exceptions.HTTPError:
            print(f"Data for {date_str} is not available. Skipping this date.")
            continue
    
    if not data_frames:
        print("No data available for the specified date range.")
        return None
    
    combined_df = pd.concat(data_frames).reset_index(drop=True)

    # for easier display
    combined_df[('Category', 'Unnamed: 0_level_1')] = combined_df[('Category', 'Unnamed: 0_level_1')].replace('> 5 years and <= 10 years', '> 5 and <= 10 years')

    return combined_df

In [12]:
def relabel(df):
    """
    concat row names with its higher level category
    """
    # Get the index positions of the higher categories (e.g., rows that are not "On-the-run" or "Off-the-run")
    higher_category_indices = df.index[~df[('Category', 'Unnamed: 0_level_1')].str.contains('On-the-run|Off-the-run', case=False, na=False)]

    # Iterate through each higher category index
    for i in higher_category_indices:
        higher_category_name = df.at[i, ('Category', 'Unnamed: 0_level_1')]
        j = i + 1
        while j < len(df) and ('On-the-run' in df.at[j, ('Category', 'Unnamed: 0_level_1')] or 'Off-the-run' in df.at[j, ('Category', 'Unnamed: 0_level_1')]):
            df.at[j, ('Category', 'Unnamed: 0_level_1')] = higher_category_name + ' ' + df.at[j, ('Category', 'Unnamed: 0_level_1')]
            j += 1

    return df


In [13]:
df = hist_trace(days=look_back)
df = relabel(df)
df.head()

Data for 2024-09-02 is not available. Skipping this date.
Data for 2024-09-19 is not available. Skipping this date.


Category ATS & Interdealer           Dealer to Customer  \
       Unnamed: 0_level_1            Trades Par Value             Trades   
0                    TIPS             880.0       5.9             1256.0   
1              <= 5 years             416.0       3.6              592.0   
2   <= 5 years On-the-run              90.0       1.8               70.0   
3  <= 5 years Off-the-run             326.0       1.8              522.0   
4     > 5 and <= 10 years             260.0       1.8              337.0   

              Total                  Date  
  Par Value  Trades Par Value              
0       9.2  2136.0      15.1  2024-08-08  
1       5.9  1008.0       9.5  2024-08-08  
2       2.2   160.0       4.0  2024-08-08  
3       3.7   848.0       5.5  2024-08-08  
4       1.8   597.0       3.6  2024-08-08

In [14]:
df[('Category', 'Unnamed: 0_level_1')].unique()

array(['TIPS', '<= 5 years', '<= 5 years On-the-run',
       '<= 5 years Off-the-run', '> 5 and <= 10 years',
       '> 5 and <= 10 years On-the-run',
       '> 5 and <= 10 years Off-the-run', '> 10 years',
       '> 10 years On-the-run', '> 10 years Off-the-run'], dtype=object)

In [15]:
tenors = [tenor for tenor in df[('Category', 'Unnamed: 0_level_1')].unique() 
          if not any(exclusion in tenor for exclusion in ['TIPS', 'On-the-run', 'Off-the-run'])]

categories = ['ATS & Interdealer', 'Dealer to Customer', 'Total']
metrics = ['Trades', 'Par Value']

category_colors = {
    'ATS & Interdealer': 'skyblue',
    'Dealer to Customer': 'orange'
}

# Helper function to add traces for Trades and Par Value, grouped by legendgroup
def add_traces(df, col_offset):

    # Add "Trades" to the first row (row=1)
    row = 1
    for category in categories[:2]:
        fig.add_trace(
            go.Bar(
                x=df['Date'], 
                y=df[(category, 'Trades')], 
                name=f'{category} Trades',
                marker_color=category_colors[category],
                legendgroup=f'{category}',  # Grouping legend entries by category
                showlegend=(col_offset == 0)  # Show legend only in the first column
            ),
            row=row, col=col_offset + 1
        )

    # Add horizontal line for the latest "Trades" value 
    if not df.empty and (categories[2], 'Trades') in df.columns:
        latest_date_value = df[df['Date'] == df['Date'].max()][(categories[2], 'Trades')].values[0]
        fig.add_trace(
            go.Scatter(
                x=[df['Date'].min(), df['Date'].max()],
                y=[latest_date_value, latest_date_value],
                mode='lines',
                line=dict(color='red', dash='dash'),
                showlegend=False
            ),
            row=row, col=col_offset + 1
        )

    # Add "Par Value" to the second row (row=2)
    row = 2
    for category in categories[:2]:
        fig.add_trace(
            go.Bar(
                x=df['Date'], 
                y=df[(category, 'Par Value')], 
                name=f'{category} Par Value',
                marker_color=category_colors[category],
                legendgroup=f'{category}',  # Grouping legend entries by category
                showlegend=(col_offset == 0)  # Show legend only in the first column
            ),
            row=row, col=col_offset + 1
        )

    # Add horizontal line for the latest "Par Value" value
    if not df.empty and (categories[2], 'Par Value') in df.columns:
        latest_date_value = df[df['Date'] == df['Date'].max()][(categories[2], 'Par Value')].values[0]
        fig.add_trace(
            go.Scatter(
                x=[df['Date'].min(), df['Date'].max()],
                y=[latest_date_value, latest_date_value],
                mode='lines',
                line=dict(color='red', dash='dash'),
                showlegend=False
            ),
            row=row, col=col_offset + 1
        )


In [16]:
df_TIPS = df[df[('Category', 'Unnamed: 0_level_1')] == "TIPS"]

fig = make_subplots(
    rows=2, cols=1, 
    subplot_titles=[
        f'Total Trades', 
        f'Total Par Value'
    ]
)

add_traces(df_TIPS, 0)  


fig.update_layout(
    height=800,
    width=1500,
    barmode='stack',
    title_text=f'TRACE Analysis for TIPS',
    showlegend=True,
    legend=dict(
        x=1.05,  
        y=1,
        font=dict(size=10)
    )
)

fig.update_xaxes(
    tickangle=-90,
    nticks=len(df_TIPS['Date']),
    tickmode="linear"
)

fig.show()


In [17]:
# Loop through each tenor (excluding TIPS) to create a grid plot: 2 rows (Trades and Par Value) x 3 columns (Tenor, On-the-run, Off-the-run)
for tenor in tenors:
    df_tenor = df[df[('Category', 'Unnamed: 0_level_1')] == tenor]
    df_ontherun = df[df[('Category', 'Unnamed: 0_level_1')] == f'{tenor} On-the-run']
    df_offtherun = df[df[('Category', 'Unnamed: 0_level_1')] == f'{tenor} Off-the-run']
    
    fig = make_subplots(
        rows=2, cols=3, 
        subplot_titles=[
            f'Total Trades', f'On-the-run Trades', f'Off-the-run Trades',
            f'Total Par Value', f'On-the-run Par Value', f'Off-the-run Par Value'
        ]
    )

    # Add traces for the tenor, on-the-run, and off-the-run plots
    add_traces(df_tenor, 0)  
    add_traces(df_ontherun, 1) 
    add_traces(df_offtherun, 2)  

  
    fig.update_layout(
        height=800,
        width=1500,
        barmode='stack',
        title_text=f'TRACE Analysis for {tenor}',
        showlegend=True,
        legend=dict(
            x=1.05,  
            y=1,
            font=dict(size=10)
        )
    )

    fig.update_xaxes(
        tickangle=-90,
        nticks=len(df_tenor['Date'])
    )

    fig.show()